In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

from langchain_groq import ChatGroq

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm = ChatGroq(model="qwen-qwq-32b")

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

## **Retriever for the LangGraph**

In [5]:
urls = [
    "https://langchain-ai.github.io/langgraph/",
    "https://modelcontextprotocol.io/introduction",
    "https://langchain-ai.github.io/langgraph/agents/multi-agent/"
]

In [10]:
docs = [WebBaseLoader(url).load() for url in urls]
len(docs)

3

In [16]:
doc_list = [item for sublist in docs for item in sublist]

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

doc_splits = text_splitter.split_documents(documents=doc_list)
print(doc_splits[1:3])

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/', 'title': 'LangGraph', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content='Deployment\n    \n  \n\n\n\n\n\n\n\n\n\n    Prebuilt agents\n    \n  \n\n\n\n\n\n            Prebuilt agents\n          \n\n\n\n\n    Overview\n    \n  \n\n\n\n\n\n    Running agents\n    \n  \n\n\n\n\n\n    Streaming\n    \n  \n\n\n\n\n\n    Models\n    \n  \n\n\n\n\n\n    Tools\n    \n  \n\n\n\n\n\n    MCP Integration\n    \n  \n\n\n\n\n\n    Context\n    \n  \n\n\n\n\n\n    Memory\n    \n  \n\n\n\n\n\n    Human-in-the-loop\n    \n  \n\n\n\n\n\n    Multi-agent\n    \n  \n\n\n\n\n\n    Evals\n    \n  \n\n\n\n\n\n    Deployment\n    \n  \n\n\n\n\n\n    UI\n    \n  \n\n\n\n\n\n\n\n\n\n    LangGraph framework\n    \n  \n\n\n\n\n\n            LangGraph framework\n          \n\n\n\n\n    Agent architectures\n    \n  \n\n\n\n\n\n\n    Graphs\n    \n  \n\n\n\n\n\n\n    Streaming\n    

## ***Use HuggingFace Embedding Model***

In [18]:
embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

c:\Users\Lenovo\anaconda3\envs\lang_aca\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
vectorstore = FAISS.from_documents(
    documents=doc_splits,
    embedding=embeddings,
)

langGraph_retriever = vectorstore.as_retriever()

In [22]:
query = "Why Lnaggraph is important?"

response = langGraph_retriever.invoke(query)
response

[Document(metadata={'source': 'https://langchain-ai.github.io/langgraph/', 'title': 'LangGraph', 'description': 'Build reliable, stateful AI systems, without giving up control', 'language': 'en'}, page_content='LangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n          Skip to content\n        \n\n\n\n\n\n\n\nWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            LangGraph\n          \n\n\n\n            \n              LangGraph\n            \n          \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            Initializing search\n          \n\n\n\n\n\n\n\n\n\n\n\n\n    GitHub\n  \n\n\n\n\n\n\n\n\n          \n  \n  \n    \n  \n  Guides\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Reference\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Examples\n\n        \n\n\n\n          \n  \n  \n    \n  \n  Resources\n\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n

## Make this retriever into a retriever tools

In [23]:
from langchain.tools.retriever import create_retriever_tool

langGraph_retriever_tool = create_retriever_tool(
    langGraph_retriever,
    "retriever_vector_db_blog",
    "Search and run information abut Model Context Protocol(MCP) and multi-agent system."
)

In [24]:
langGraph_retriever_tool

Tool(name='retriever_vector_db_blog', description='Search and run information abut Model Context Protocol(MCP) and multi-agent system.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001FD043F6C00>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FD7386DC50>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001FD290658A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FD7386DC50>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_typ

# **LangChain blogs - Separate Vector DB**

In [25]:
urls = [
    "https://python.langchain.com/docs/tutorials/",
    "https://python.langchain.com/docs/introduction/",
    "https://python.langchain.com/docs/versions/v0_3/"   
]

In [26]:
docs = [WebBaseLoader(url).load() for url in urls]
len(docs)

3

In [ ]:
doc_list = [item for sublist in docs for item in sublist]

3

In [30]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=200
)

doc_splits = text_splitter.split_documents(documents=doc_list)

vectorstore_langChain = FAISS.from_documents(
    documents=doc_splits,
    embedding=embeddings
)

In [31]:
langChain_retriever = vectorstore_langChain.as_retriever()

langChain_retriever.invoke("Why langChain?")

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/versions/v0_3/', 'title': 'LangChain v0.3 | 🦜️🔗 LangChain', 'description': 'Last updated: 09.16.24', 'language': 'en'}, page_content='LangChain v0.3 | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/tutorials/', 'title': 'Tutorials | 🦜️🔗 LangChain', 'description': 'New to LangChain or LLM app development in general? Read this material to quickly get up and running building your first applications.', 'language': 'en'}, page_content='Tutorials | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 

In [32]:
langChain_retriever_tool = create_retriever_tool(
    retriever=langChain_retriever,
    name="Retriever_vector_langchain_blog",
    description="Search and run information about langchain tutorial and different version."
)

In [33]:
langChain_retriever_tool

Tool(name='Retriever_vector_langchain_blog', description='Search and run information about langchain tutorial and different version.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001FD043F6C00>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FD705BFED0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types={}, partial_variables={}, template='{page_content}'), document_separator='\n\n', response_format='content'), coroutine=functools.partial(<function _aget_relevant_documents at 0x000001FD290658A0>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FD705BFED0>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page_content'], input_types

In [34]:
tools = [langChain_retriever_tool, langGraph_retriever_tool]